In [1]:
import numpy as np
import os
import pickle
import random

import import_ipynb
from constants import NUM_FBANK, HOP_LENGTH, NUM_PER_SPEAKER, NUM_FRAME

importing Jupyter notebook from constants.ipynb


In [20]:
def generate_val_dataset(input_dir, output_dir):
    NUM_PER_SPEAKER = 100 # 원래 300인데 이만큼의 길이가 안나오는 화자가 많아서 100으로 일단 줄임
    
    # 등록 발화 5, true 발화 5, false 발화 10
    
    X = []
    for speaker in os.listdir(input_dir):
        X_speaker = []
        for utt in os.listdir(input_dir+'/'+speaker):
            spec = np.load(input_dir+'/'+speaker+'/'+utt)
            for i in range(0,spec.shape[0]-NUM_FRAME+1,NUM_FRAME): X_speaker.append(spec[i:i+NUM_FRAME,:])
            if len(X_speaker)>=NUM_PER_SPEAKER: break
        if len(X_speaker)<NUM_PER_SPEAKER: continue
        X.append(np.array(X_speaker[:NUM_PER_SPEAKER]))
    X = np.array(X)
    num_speaker = len(X)
    print('number of speakers who has more than minimun:', num_speaker)
    
    # sample utterances for validation
    val_X = []
    for speaker in range(num_speaker):
        enroll_true_utt = X[speaker][np.random.choice(NUM_PER_SPEAKER, 10, replace=False)]  # enroll 발화 5개, true 발화 5개
        val_X.append(enroll_true_utt)
        
        others = np.random.choice(num_speaker-1, 10, replace=False)
        false_utt = []
        for i in range(10):
            if others[i]==speaker: others[i]=num_speaker-1
            false_utt.append(X[others[i]][np.random.choice(NUM_PER_SPEAKER,1)[0]])
        false_utt = np.array(false_utt)
        val_X.append(false_utt)
    val_X = np.concatenate(val_X)
    print('total generated shape:',val_X.shape)
    
    val_Y = np.zeros(len(val_X)) # dummy

    # save
    XY = (val_X, val_Y)
    with open(output_dir+'/'+f"{num_speaker}.npy",'wb') as f:
        pickle.dump(XY,f)

In [21]:
generate_val_dataset('C:/Users/LeeJunghun/Desktop/lingometer/data/NIKL_DIALOGUE_2020_PCM_v1.2_part1/NIKL_DIALOGUE_2020_PCM_v1.2_part1/npy/validation', 'C:/Users/LeeJunghun/Desktop/lingometer/data/NIKL_DIALOGUE_2020_PCM_v1.2_part1/NIKL_DIALOGUE_2020_PCM_v1.2_part1/validation_batcher')

number of speakers who has more than minimun: 64
total generated shape: (1280, 128, 128, 1)
